In [ ]:
import pandas as pd
import re
import string
import torch
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/movies.csv' )
df.head()

,ro,fa,source
0,Deschide-ti ochii.,چشماتو باز کن,encanto
1,De aici provine magia noastra?,جادوی ما از اینجا میاد؟,encanto
2,Aceasta lumanare detine miracolul\ncare i-a fo...,این شمع معجزه داده شده\nبه خانواده ما رو نگه د...,encanto
3,Cum am primit un miracol?,چطوری یه معجزه رو گرفتیم؟,encanto
4,"Cu mult timp in urma,",خیلی وقت پیش...,encanto


In [ ]:

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.15, random_state = 5)

In [ ]:
!pip install transformers[sentencepiece]
#using pretrained model and then finetunig it on our dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-mul-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-mul-en")

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr=0.0001)

In [ ]:
def model_train():
    model.train()
    losses = 0
    X = df['fa']
    y = df['ro']
    max_epochs = 27
    n_batches = 32
    for epoch in tqdm(range(max_epochs)):
        for i in tqdm(range(125)):
            # making batches 
            local_X, local_y = X[i*n_batches:(i+1)*n_batches,], y[i*n_batches:(i+1)*n_batches,]
            # preparing the data according to the model input
            batch = tokenizer.prepare_seq2seq_batch(list(local_X),list(local_y),return_tensors='pt')
            output = model(**batch)
            # loss can be taken directly from the model output
            loss = output.loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses = losses+loss
    average = losses/len(df)
    print('Loss: ' + str(average) )
    
    return model

In [ ]:
if Train:
    model = model_train()

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3426: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


ValueError: ignored

In [ ]:
a = model.generate(**tokenizer.prepare_seq2seq_batch('Nínú ìpè kan lẹ́yìn ìgbà náà, wọ́n sọ fún aṣojú iléeṣẹ́ BlaBlaCar pé ètò náà ti yí padà, pé',return_tensors='pt').to('cuda'))
text = tokenizer.batch_decode(a)

In [ ]:
text = str(text)
text = re.sub("<pad> ","",text)
text = re.sub("'","",text)
text = text.replace("[", "")
text = text.replace("]", "")

In [ ]:
test = pd.read_csv("../input/yorb-machine-translation/Test.csv")

In [ ]:
test["Label"] = ""
test["Label"]=test["fa"].progress_apply(lambda x:tokenizer.batch_decode(model.generate(**tokenizer.prepare_seq2seq_batch([x],return_tensors='pt'))))

In [ ]:
test['Label'] = test['Label'].astype(str)
test['Label'] = test['Label'].apply(lambda x: re.sub("<pad> ","",x))
test['Label'] = test['Label'].apply(lambda x: re.sub("'","",x))
test['Label'] = test['Label'].apply(lambda x: x.replace("[", ""))
test['Label'] = test['Label'].apply(lambda x: x.replace("]", ""))
test['Label'] = test['Label'].apply(lambda x: x.replace('"', ""))

In [ ]:


import fractions
import math
from collections import Counter

from nltk.util import ngrams


def sentence_nist(references, hypothesis, n=5):
    hypothesis1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which',
                  'ensures', 'that', 'the', 'military', 'always',
                  'obeys', 'the', 'commands', 'of', 'the', 'party']

    hypothesis2 = ['It', 'is', 'to', 'insure', 'the', 'troops',
                   'forever', 'hearing', 'the', 'activity', 'guidebook',
                   'that', 'party', 'direct']

    reference1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
                   'ensures', 'that', 'the', 'military', 'will', 'forever',
                   'heed', 'Party', 'commands']

    reference2 = ['It', 'is', 'the', 'guiding', 'principle', 'which',
                   'guarantees', 'the', 'military', 'forces', 'always',
                   'being', 'under', 'the', 'command', 'of', 'the',
                   'Party']

    reference3 = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
                  'army', 'always', 'to', 'heed', 'the', 'directions',
                   'of', 'the', 'party']

    sentence_nist([reference1, reference2, reference3], hypothesis1) 


    sentence_nist([reference1, reference2, reference3], hypothesis2)
    

In [ ]:
from nltk.translate import nist_score, bleu_score
hypothesis1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which','ensures', 'that', 'the', 'military', 'always','obeys', 'the', 'commands', 'of', 'the', 'party']
reference1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that','ensures', 'that', 'the', 'military', 'will', 'forever','heed', 'Party', 'commands']
bleu_score.sentence_bleu([reference1], hypothesis1, smoothing_function=bleu_score.SmoothingFunction().method3)

0.41180376356915777

In [ ]:
nist_score.sentence_nist([reference1], hypothesis1)

0.05876787197888857

In [ ]:
# Natural Language Toolkit: ChrF score
#
# Copyright (C) 2001-2021 NLTK Project
# Authors: Maja Popovic
# Contributors: Liling Tan, Aleš Tamchyna (Memsource)
# URL: <https://www.nltk.org/>
# For license information, see LICENSE.TXT

""" ChrF score implementation """
import re
from collections import Counter, defaultdict

from nltk.util import ngrams


def sentence_chrf(
    reference, hypothesis, min_len=1, max_len=6, beta=3.0, ignore_whitespace=True
):
 
    return corpus_chrf(
        [reference],
        [hypothesis],
        min_len,
        max_len,
        beta=beta,
        ignore_whitespace=ignore_whitespace,
    )



def _preprocess(sent, ignore_whitespace):
    if type(sent) != str:
        # turn list of tokens into a string
        sent = " ".join(sent)

    if ignore_whitespace:
        sent = re.sub(r"\s+", "", sent)
    return sent


def chrf_precision_recall_fscore_support(
    reference, hypothesis, n, beta=3.0, epsilon=1e-16
):
    ref_ngrams = Counter(ngrams(reference, n))
    hyp_ngrams = Counter(ngrams(hypothesis, n))

    # calculate the number of ngram matches
    overlap_ngrams = ref_ngrams & hyp_ngrams
    tp = sum(overlap_ngrams.values())  # True positives.
    tpfp = sum(hyp_ngrams.values())  # True positives + False positives.
    tpfn = sum(ref_ngrams.values())  # True positives + False negatives.

    try:
        prec = tp / tpfp  # precision
        rec = tp / tpfn  # recall
        factor = beta ** 2
        fscore = (1 + factor) * (prec * rec) / (factor * prec + rec)
    except ZeroDivisionError:
        prec = rec = fscore = epsilon
    return prec, rec, fscore, tp



def corpus_chrf(
    references, hypotheses, min_len=1, max_len=6, beta=3.0, ignore_whitespace=True
):
    assert len(references) == len(
        hypotheses
    ), "The number of hypotheses and their references should be the same"
    num_sents = len(hypotheses)

    # Keep f-scores for each n-gram order separate
    ngram_fscores = defaultdict(lambda: list())

    # Iterate through each hypothesis and their corresponding references.
    for reference, hypothesis in zip(references, hypotheses):

        # preprocess both reference and hypothesis
        reference = _preprocess(reference, ignore_whitespace)
        hypothesis = _preprocess(hypothesis, ignore_whitespace)

        # Calculate f-scores for each sentence and for each n-gram order
        # separately.
        for n in range(min_len, max_len + 1):
            # Compute the precision, recall, fscore and support.
            prec, rec, fscore, tp = chrf_precision_recall_fscore_support(
                reference, hypothesis, n, beta=beta
            )
            ngram_fscores[n].append(fscore)

    # how many n-gram sizes
    num_ngram_sizes = len(ngram_fscores)

    # sum of f-scores over all sentences for each n-gram order
    total_scores = [sum(fscores) for n, fscores in ngram_fscores.items()]

    # macro-average over n-gram orders and over all sentences
    return (sum(total_scores) / num_ngram_sizes) / num_sents

In [ ]:
ref1 = str('It is a guide to action that ensures that the military will forever heed Party commands').split()
hyp1 = str('It is a guide to action which ensures that the military always obeys the commands of the party').split()
hyp2 = str('It is to insure the troops forever hearing the activity guidebook that party direct').split()
sentence_chrf(ref1, hyp1)

0.6349903001842703

In [ ]:
sentence_chrf(ref1, hyp2)

0.3330122858264257